# Нечеткие автоматы

In [ ]:
import numpy as np
import torch

In [ ]:
import sys
sys.path.insert(0, './python')

In [ ]:
from fuzzy_torch import logic
from fuzzy_torch.modules import ffsa, indicators

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

font = {'family' : 'Liberation Sans',
        'weight' : 'normal',
        'size'   : 30}

matplotlib.rc('font', **font)

## Проверка переходов

In [ ]:
Logic = logic.Hamacher
fuzzy_fsa = ffsa.TimeIndependentFFSA(Logic)

In [ ]:
# Добавление состояний и переходов.
fuzzy_fsa.states = [0, 1]
fuzzy_fsa.transitions.append(ffsa.FuzzyTransition(0, 1, indicators.Sigmoid(1, 1.0, 0.0)))

In [ ]:
# Начальные активации.
activation = torch.tensor(np.array([[1.0, 0.0],
                                    [1.0, 0.0],
                                    [1.0, 0.0],
                                    [1.0, 0.0]]).astype(np.float32), requires_grad=True)

# Последовательности.
sequence = torch.tensor(np.array([[-1.0, -1.0, 0.0, 1.0, 1.0],
                                  [-10.0, -5.0, -1.0, 0.0, 10.0],
                                  [-10.0, -10.0, -10.0, -10.0, 0.0],
                                  [-0.5, -0.5, -0.5, -0.5, -0.5]]).astype(np.float32), requires_grad=True)

In [ ]:
Activations = [activation]
for step in range(sequence.size()[1]):
    Activations.append(fuzzy_fsa(sequence[:, step][:,None], Activations[-1]))

In [ ]:
torch.sum(Activations[-1][:,1]).backward()

In [ ]:
Activations

In [ ]:
sequence.grad

In [ ]:
print(fuzzy_fsa.transitions[0].condition.linear.weight.grad)
print(fuzzy_fsa.transitions[0].condition.linear.bias.grad)

## Последовательность с переключением

In [ ]:
class SwitchingRegressor(torch.nn.Module):
    def __init__(self, logic, ffsa):
        super().__init__()
        self.logic = logic
        self.ffsa = ffsa
        self.debug = False
        
    def forward(self, input, init_activation):
        steps = input.size()[1]
        
        activations = [init_activation]
        outputs = []
        for step in range(steps):
            # Срез входа по текущему шагу.
            input_on_current_step = input[:, step]
            
            # Новые активации (согласно нечеткому конечному автомату).
            activations.append(self.ffsa(input_on_current_step, activations[-1]))
            
            # Получение выходов регрессоров.
            output = [state(input_on_current_step) for state in self.ffsa.states]
            output = torch.stack(output, dim=1)
            outputs.append(torch.einsum("bo,bo...->b...", activations[-1], output))
            
        return torch.stack(outputs, dim=1), torch.stack(activations, dim=1)

In [ ]:
# Логика.
#Logic = logic.Godel
#Logic = logic.Product
#Logic = logic.Lukasiewicz
#Logic = logic.Nilpotent
Logic = logic.Hamacher

In [ ]:
regressor = SwitchingRegressor(Logic, ffsa.TimeIndependentFFSA(Logic, normalize=True))

In [ ]:
regressor.ffsa.states = torch.nn.ModuleList([torch.nn.Linear(2, 1), torch.nn.Linear(2, 1), torch.nn.Linear(2, 1)])
regressor.ffsa.transitions = torch.nn.ModuleList([ffsa.FuzzyTransition(0, 1, indicators.Sigmoid(2)),
                                                  ffsa.FuzzyTransition(0, 2, indicators.Sigmoid(2))])

### Набор данных

In [ ]:
import scipy.stats as stats

class SwitchingSequences(torch.utils.data.Dataset):
    def __init__(self, length=100, delta=3):       
        self.length = length
        self.delta = delta
        self.noize = stats.norm()
    
    def __len__(self):
        return 16384
    
    def __getitem__(self, idx):
        switch_index_start = np.random.choice(np.arange(0, self.length - self.delta - 1), 1)[0]
        switch_index_end = np.random.choice(np.arange(switch_index_start + self.delta, self.length), 1)[0]
        switch_type = np.random.choice([-1.0, 1.0], 1)[0]
        
        X = np.ones((self.length, 2))
        X[:,1] *= switch_type
        X[:switch_index_start, 1] = 0.0
        X[switch_index_end:, 1] = 0.0
        X += self.noize.rvs(X.shape) * 0.1
        
        y = np.ones((self.length))
        y[switch_index_start:] = 2.0 * switch_type * X[switch_index_start:, 0]
        y += self.noize.rvs(y.shape) * 0.1
        
        return X.astype(np.float32), y.astype(np.float32)[:, None]

In [ ]:
dataset = SwitchingSequences()

In [ ]:
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=512,
    shuffle=False,
    num_workers=0,
    collate_fn=None,
    pin_memory=False,
 )

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(regressor.parameters(), lr=0.1)

In [ ]:
loss = torch.nn.MSELoss()

for epoch in range(20):
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        x, true_y = data
        init_activations = torch.zeros(x.size()[0], 3)
        init_activations[:,0] = 1.0

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        y, activations = regressor(x, init_activations)
        eval_loss = loss(y, true_y)
        eval_loss.backward()
        optimizer.step()

        # print statistics
        running_loss += eval_loss.item()
        print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss:.3f}')
        running_loss = 0.0

print('Finished Training')

In [ ]:
X, y_true = dataset[0]
X = torch.tensor(X)[None,:]
y, activations = regressor(X, torch.tensor(np.array([[1.0, 0.0, 0.0]]).astype(np.float32)))

print(y_true.squeeze())
print(y.squeeze().detach().numpy())

In [ ]:
activations.argmax(dim=2)

In [ ]:
activations.sum(dim=2)

In [ ]:
print("Первое условие (параметры):")
print(regressor.ffsa.transitions[0].condition.linear.weight.detach().numpy())
print(regressor.ffsa.transitions[0].condition.linear.bias.detach().numpy())

print("Второе условие (параметры):")
print(regressor.ffsa.transitions[1].condition.linear.weight.detach().numpy())
print(regressor.ffsa.transitions[1].condition.linear.bias.detach().numpy())

In [ ]:
fig = plt.figure()
ax = fig.gca()

fig.set_figheight(12)
fig.set_figwidth(24)
ax.grid(color='#000000', alpha=0.15, linestyle='-', linewidth=1, which='major')
ax.grid(color='#000000', alpha=0.1, linestyle='-', linewidth=0.5, which='minor')

ax.set_xlabel('Время')
ax.set_ylabel('Значение')

T = np.arange(dataset.length)
ax.plot(T, X[0,:], label="X(t)")
ax.plot(T, y_true, label="y(t)")
#ax.plot(T, y[0].detach().numpy(), label="y'(t)")

ax.legend(loc='upper left')